In [3]:
from flask import Flask, request, jsonify
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
import joblib
import os

In [5]:
# Inisialisasi Flask
app = Flask(__name__)

# Load model SVM yang sudah dilatih
svm_model = joblib.load("best_svm_model.pkl")

# Load MobileNetV2 tanpa fully connected layer (untuk ekstraksi fitur)
mobilenet = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(128,128,3))


# Label kelas (ubah sesuai dataset kamu)
class_labels = ["bersih", "kotor"]
def extract_feature_single(img_path):
    """Ekstraksi fitur satu gambar (ukuran 128x128)"""
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # normalisasi MobileNetV2
    features = mobilenet.predict(img_array, verbose=0)
    return features.reshape(1, -1)
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'Tidak ada file gambar dikirim'}), 400

    file = request.files['image']
    file_path = os.path.join('temp_image.jpg')
    file.save(file_path)

    try:
        # Ekstraksi fitur dari gambar
        features = extract_feature_single(file_path)

        # Prediksi dengan model SVM
        prediction = svm_model.predict(features)[0]
        probabilities = None
        if hasattr(svm_model, "decision_function"):
            probabilities = svm_model.decision_function(features).tolist()

        # Ambil nama label prediksi
        result_label = class_labels[int(prediction)] if isinstance(prediction, (int, np.integer)) else prediction

        return jsonify({
            'prediction': result_label,
            'raw_prediction': str(prediction),
            'probabilities': probabilities
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        if os.path.exists(file_path):
            os.remove(file_path)

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1